In [1]:
import numpy as np
import pandas as pd
from IPython.display import Markdown, display

# edf libraries
import pyedflib
import mne


#pca dependencies
from sklearn import decomposition

#direction estimation class dependencies
import pyargus
import pyargus.directionEstimation as pde
from pyargus.directionEstimation import *




C:\Users\marit\Anaconda3\lib\site-packages\numba\decorators.py:146: RuntimeWarning: Caching is not available when the 'parallel' target is in use. Caching is now being disabled to allow execution to continue.
  warnings.warn(msg, RuntimeWarning)


In [2]:
def printmd(string):
    display(Markdown(string))

In [3]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines){
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
printmd("# Feature Extraction from EEG Data - Output to .NPY files")
printmd(""" This notebook reads raw EEG data from EDF files, selects as subset of channels and extracts features for each
            of those channels. Data is saved to numpy (.NPY) files for later use. """)
printmd("Please note that patient s07 has been removed due to poor quality. The first and last 120s of data have been removed.")
printmd("Principal Component Analysis is used as the primary method of reducing noise.")


        

# Feature Extraction from EEG Data - Output to .NPY files

 This notebook reads raw EEG data from EDF files, selects as subset of channels and extracts features for each
            of those channels. Data is saved to numpy (.NPY) files for later use. 

Please note that patient s07 has been removed due to poor quality. The first and last 120s of data have been removed.

Principal Component Analysis is used as the primary method of reducing noise.

### &emsp; &emsp; Feature Extraction Methods (https://www.hindawi.com/journals/isrn/2014/730218/)


- Time Frequency Distributions (TFD) <br><br>
- Fast Fourier Transform (FFT) <br><br>
- Eigenvector Methods (EM), 
- - Pisarenko’s Method -- Pisarenko Harmonic Decomposition --try https://dsp.stackexchange.com/questions/7667/pisarenko-harmonic-decomposition
- - MUSIC Method -- https://pypi.org/project/pyargus/
- - Minimum Norm Method <br><br>
- Wavelet Transform (WT)
- - Continuous Wavelet Transform (CWT) Method
- - Discrete Wavelet Transform (DWT) <br><br>
    
- Auto regressive method (ARM)
- - Yule-Walker Method --https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.yule_walker.html
- - Burg’s Method <br><br>
    
More possibilities: https://kourouklides.fandom.com/wiki/Statistical_Signal_Processing
    

In [5]:
# from EEGNET Implementation notebook

ignore_list = ['s07']  #list of patient files that should be skipped
sample_size = 20

import pyedflib
mne.set_log_level("WARNING")

# get the minimum length of the files
def get_minimum_duration(group_directory_name, patient_group_file_prefix):
    file_durations = []
    for i in range (1, 15): # reading 14 files
        patient_id = "{}{:02d}".format(patient_group_file_prefix, i)
        file_name = '{}\\{}.edf'.format(group_directory_name, patient_id)
        f = pyedflib.EdfReader(file_name)
        file_durations.append(f.file_duration)
        f.close()
    return(min(file_durations))

# modified based on https://stackoverflow.com/a/48704557/2466781
def chunk(seq, size):
    sl = len(seq) - (len(seq)%size) #exclude values that will be out of range
    r = [pd.DataFrame(seq[pos:pos + size]) for pos in range(0, sl, size)]
    return r

# Modified version of process_patient_group in older notebooks
# Uses the raw EDF files and converts to dataframe, dropping the first and last 120 seconds of the shortest  file
# All other files are trimmed similarly to produce the same size
# Adapted from page 1 of https://buildmedia.readthedocs.org/media/pdf/pyedflib/latest/pyedflib.pdf
def process_patient_group(group_directory_name, patient_group_file_prefix, 
                          minimum_original_duration, 
                          plot_channels = False,
                         channels = ['F8', 'F7', 'F4', 'F3', 'Fz']):
    meta_df = pd.DataFrame()
    meta = []
    patient_id_list = []
    for i in range (1, 15): # reading 14 files
        patient_id = "{}{:02d}".format(patient_group_file_prefix, i)
        patient_id_list.append(patient_id)
        
        file_name = '{}\\{}.edf'.format(group_directory_name, patient_id)
        data = mne.io.read_raw_edf(file_name)
        df = data.to_data_frame()
        df2 = df[channels]
        ## drop the first 120 seconds and last 120 seconds
        df2 = df2[120: (minimum_original_duration-120)]
        #f = pyedflib.EdfReader(file_name)
        #f.close()
        if patient_id not in ignore_list:
            meta_df = meta_df.append(df2)
            
    batches = chunk(meta_df, sample_size)

    for batch in batches:
        #display(np.asarray(batch.values).shape)
        meta.append([np.asarray(batch.values,dtype=np.float32)])
           
                    
    return meta
        
    

In [6]:
# Retrieve patient data, using a time window determined by the shortest recording
# patient s07 is removed

all_channels = ['Fp2', 'F8', 'T4', 'T6', 'O2', 'Fp1', 'F7', 'T3', 'T5', 'O1', 'F4',
                'C4', 'P4', 'F3', 'C3', 'P3', 'Fz', 'Cz', 'Pz']
target_channels = ['T4', 'T6', 'O2', 'T3', 'T5', 'O1',
                   'C4', 'P4', 'C3', 'P3', 'Cz', 'Pz']

minimum_duration = min(get_minimum_duration("Healthy Controls", "h"), get_minimum_duration('SZ Patients', 's'))
print('Retrieving raw data')
print('Minimum file duration: {} seconds'.format( minimum_duration))
print("Healthy Controls")
hc_data = process_patient_group('Healthy Controls', 'h', minimum_duration, channels=target_channels)
display(np.asarray(hc_data).shape)


print('Sz Patients')
sz_data = np.asarray(process_patient_group('SZ Patients', 's', minimum_duration, channels=target_channels), dtype=np.float32)
display(np.asarray(sz_data).shape)



Retrieving raw data
Minimum file duration: 740 seconds
Healthy Controls


(350, 1, 20, 12)

Sz Patients


(325, 1, 20, 12)

In [7]:
import scipy

In [16]:
%matplotlib qt5
%matplotlib inline
mne.set_log_level("WARNING")


from scipy.fftpack import dct, idct
    
# adapted from: https://docs.scipy.org/doc/scipy/reference/tutorial/fftpack.html#type-iii-dct
def fast_fourier_transform(x, dct_type=2):
    #norm = 'None' | 'ortho'
    # get discreet cosine transforms
    fft_dct = dct(dct(x, type=dct_type, norm='ortho'), type=3, norm='ortho')
    # get inverse discreet cosine transforms
    fft_idct = idct(dct(x, type=dct_type), type=2)
    return fft_dct, fft_idct


    
# Yule-Walker Method -- https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.yule_walker.html
# A paper arguing against this method: http://stat.wharton.upenn.edu/~steele/Courses/956/Resource/YWSourceFiles/WhyNotToUseYW.pdf
def yule_walker_method(X):
    import statsmodels.api as sm
    #from statsmodels.datasets.sunspots import load
    
    #Y = sm.add_constant(X)
    #data = load(as_pandas=False)
    rho, sigma = sm.regression.yule_walker(X, order=len(X), method="mle")
    #return rho: (AR(p) coefficients | and sigma: the estimated residual stdev
    return rho, sigma 

# http://thomas-cokelaer.info/software/spectrum/html/user/ref_param.html
# paper with implementation guidelines (no library): https://www.opus-codec.org/docs/vos_fastburg.pdf
def burgs_method(X):
    from pylab import plot, log10, linspace, axis
    #from spectrum import *
    AR, P, k = arburg(Y, shape(X)-1) # order must be less than len(Y)
    PSD = arma2psd(AR, sides='centerdc')
    #plot(linspace(-0.5, 0.5, len(PSD)), 10*log10(PSD/max(PSD)))
    #axis([-0.5,0.5,-60,0])
    # return the Burg estimates and the Power Spectral Density
    return AR, PSD





def music(X,  d=0.5):
    import pyargus
    import pyargus.directionEstimation as pde
    
    
    #from pyargus.directionEstimation import *
    ## R matrix calculation
    M=X.shape[0]
    N = 2**12  # sample size          
    theta = 90 # Incident angle of the test signal
    
    # Array response vectors of the test signal
    a = X #pd.DataFrame(X)#np.exp(np.arange(0,M,1)*1j*2*np.pi*d*np.cos(np.deg2rad(theta)))
       
    # Generate multichannel test signal 
    soi = np.random.normal(0,1,N)  # Signal of Interest
    soi_matrix  = np.outer( soi, a).T 
    
    # Generate multichannel uncorrelated noise
    noise = np.random.normal(0,np.sqrt(10**-10),(M,N))
    
    # Create received signal
    rec_signal = soi_matrix + noise
    
    ## R matrix calculation
    R = pde.corr_matrix_estimate(rec_signal.T, imp="mem_eff")
    # Generate scanning vectors
    array_alignment = np.arange(0, M, 1) * d
    incident_angles= np.arange(0,len(X),1) #modified result set size from static value of 81
    ula_scanning_vectors = pde.gen_ula_scanning_vectors(array_alignment, incident_angles)
      
    # DOA estimation           
    #Bartlett= DOA_Bartlett(R, ula_scanning_vectors)    
    #Capon = DOA_Capon(R, ula_scanning_vectors)
    #MEM = DOA_MEM(R, ula_scanning_vectors)
    #LPM = DOA_LPM(R, ula_scanning_vectors, element_select = 0)
    MUSIC = pde.DOA_MUSIC(R, ula_scanning_vectors, signal_dimension = 1)
    return MUSIC




class direction_estimation_methods(object):
    def retrieve_features(self, extraction_method, X,  d=0.5):

        ## R matrix calculation
        M=X.shape[0]
        N = 2**12  # sample size          
        theta = 90 # Incident angle of the test signal

        # Array response vectors of the test signal
        a = X #pd.DataFrame(X)#np.exp(np.arange(0,M,1)*1j*2*np.pi*d*np.cos(np.deg2rad(theta)))

        # Generate multichannel test signal 
        soi = np.random.normal(0,1,N)  # Signal of Interest
        soi_matrix  = np.outer( soi, a).T 

        # Generate multichannel uncorrelated noise
        noise = np.random.normal(0,np.sqrt(10**-10),(M,N))

        # Create received signal
        rec_signal = soi_matrix + noise

        ## R matrix calculation
        R = pde.corr_matrix_estimate(rec_signal.T, imp="mem_eff")
        # Generate scanning vectors
        array_alignment = np.arange(0, M, 1) * d
        incident_angles= np.arange(0,len(X),1) #modified result set size from static value of 81
        ula_scanning_vectors = pde.gen_ula_scanning_vectors(array_alignment, incident_angles)

        # DOA estimation           
        # 
        
        
        methods = {
            
            'Bartlett': DOA_Bartlett(R, ula_scanning_vectors) ,
            # Capon is a high resolution nonparametric method used in 3d feature extraction
            'Capon': DOA_Capon(R, ula_scanning_vectors),
             #Linear prediction model - popular in speech recognition
            'LPM': DOA_LPM(R, ula_scanning_vectors, element_select = 0),
           
            #Burgs maximum entropy model
            'MEM': DOA_MEM(R, ula_scanning_vectors),
            
            'MUSIC': DOA_MUSIC(R, ula_scanning_vectors, signal_dimension = 1)
            
        }
        if extraction_method in methods:
            ef = methods[extraction_method]
            extracted_features = (ef * np.conj(ef)).astype(np.float32)
        else: #return all
            ef = methods
            for m in methods:
                ef[m] =  (ef[m] * np.conj(ef[m])).astype(np.float32)
                extracted_features = ef
        #multiply complex number by its conjugate to get real number
        
       
        
        

        return extracted_features



def test():
    # read data from single sz patient
    data = pd.read_csv("EEG in Schizophrenia\\SZ Patients\\s10.csv")
    data.sort_values('Unnamed: 0')
    data = data[['s08']]

    

In [9]:

test = np.linspace(-1, 1, 30, endpoint=False)
#test_output = music(test)
#display(test_output)

#display((test_output * np.conj(test_output)).astype(np.float32))
#(test_output).astype(np.float32)
#np.conj(test_output)




In [10]:
#from pyargus.directionEstimation import *

# pisarenko harmonic decomposition
# https://tkf.github.io/2010/10/03/estimate-frequency-using-numpy.html

PI = np.pi
class Pisarenko_HD(object):
       


    def covariance(self, x, k):
        N = len(x) - k
        return (x[:-k] * x[k:]).sum() / N


    def phd1(self, x):
        """Estimate frequency using Pisarenko Harmonic Decomposition"""
        r1 = self.covariance(x, 1)
        r2 = self.covariance(x, 2)
        a = (r2 + np.sqrt(r2 ** 2 + 8 * r1 ** 2)) / 4 / r1
        if a > 1:
            a = 1
        elif a < -1:
            a = -1
        return np.arccos(a)


    def freq(self, x, sample_step=3, dt=1.0):
        """Estimate frequency using `phd1`"""
        omega = self.phd1(x[::sample_step])
        display(x[::sample_step])
        display('omega ', omega)
        return omega / 2.0 / PI / sample_step / dt
    
    
    


In [11]:

    
#https://github.com/takuti/datadog-anomaly-detector/blob/5f3f4d772f344be23fef7986e081db4dcc1029c4/core/changefinder/utils.py#L51-L92
def arburg(x, k):
    """MATLAB implementation of the Burg's method.
    cf. https://searchcode.com/codesearch/view/9503568/
    """

    def sumsq(x):
        return np.sum(x * x)

    n = x.size
    # v = sumsq(x)

    # f and b are the forward and backward error sequences
    f = x[1:n]
    b = x[:(n - 1)]

    a = np.array([])

    # remaining stages i=2 to p
    for i in range(k):

        # get the i-th reflection coefficient
        denominator = sumsq(f) + sumsq(b)
        g = 0 if denominator == 0 else 2 * np.sum(f * b) / denominator

        # generate next filter order
        if i == 0:
            a = np.array([g])
        else:
            a = np.append(g, a - g * a[:(i)][::-1])

        # keep track of the error
        # v = v * (1 - g * g)

        # update the prediction error sequences
        old_f = np.empty_like(f)
        old_f[:] = f
        f = old_f[1:(n - i - 1)] - g * b[1:(n - i - 1)]
        b = b[:(n - i - 2)] - g * old_f[:(n - i - 2)]

    return -a[:k][::-1]
    


In [12]:
### Wavelet Transforms
## paper with nice review of wavelets and wavelet families: https://dergipark.org.tr/en/download/article-file/433655

from scipy import signal
import matplotlib.pyplot as plt

# adapted from https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.signal.cwt.html
# alternative implementation https://docs.obspy.org/tutorial/code_snippets/continuous_wavelet_transform.html
# parameters: X array  - input signal
#             show_plot Boolean optional - whether or not to display inline plot of transformed signal
#             widths - an array of scales by which to stretch the wavelets -  https://stackoverflow.com/questions/28822327/units-of-widths-argument-to-scipy-signal-cwt-function
# returns an array representing the estimate
def continuous_wt(X, show_plot=False, widths=None):
    if 1 in X.shape :
        X = X.flatten()
    if widths is None:
        widths = np.arange(1, len(X) +1)
        widths = np.arange(1, 2)
    sig  = np.cos(2 * np.pi * 7 * X) + signal.gausspulse(X - 0.4, fc=2)
    cwtmatr = signal.cwt(sig, signal.ricker, widths)
    #display('cwtmatr: ', cwtmatr.shape)
    if show_plot:
        plt.imshow(cwtmatr, extent=[-1, 1, 1, 50], cmap='PRGn', aspect='auto', 
                   vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max())
        plt.show()
    return cwtmatr

import pywt
# source : https://pywavelets.readthedocs.io/en/latest/ref/dwt-discrete-wavelet-transform.html
# parameters: X array wavelet, 
#             str built-in wavelet name (use wavelist() to get list of all), 
#             str mode (optional) use pywt.Modes.modes or see https://pywavelets.readthedocs.io/en/latest/ref/signal-extension-modes.html#ref-modes
# returns a tuple representing [estimate, coefficients]
def discrete_wt(X, wavelet_name='db1', mode=None):
    if mode is None:
        r = pywt.dwt(X, wavelet_name)
    else:
        r = pywt.dwt(X, wavelet_name, mode=mode)
    return r


        


In [13]:
import statsmodels.api as sm
def ordinary_least_squares(X):
    #sm.OLS(spector_data.endog, spector_data.exog)
    print(len(X))
    Y = list(range(0, len(X)))
    #print(len(Y))
    #s = sm.regression.recursive_ls(Y, X)#Poisson.OLS(Y,X)
    mod = sm.OLS(Y, X)
    
    res = mod.fit()
    s = res.params
    
    return s
#ordinary_least_squares(data)

C:\Users\marit\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [19]:
#%%time
#takes nd arrays of patient data and creates array of extracted features for use in CNN and machine learning
def extract_features(patient_data):
    all_features = []
    for entry in patient_data:
        sample_features = []
        # transpose the array so that each row represents a channel (instead of a slice of time)
        entry_t = entry[0].transpose() 
        pca = decomposition.PCA()
        #noise reduction
        #pca.fit(entry[0])
        #denoised_data = pca.transform(entry[0])
        data = entry[0]
        for channel_data in data:
            t = channel_data
            #extract features
            fft, inverse_fft = fast_fourier_transform(t)
            ywm, ywm_res_stdev = yule_walker_method(t)
            pisarenko_hd = Pisarenko_HD()
            #music_features = (music(t)).astype(float)
            dem = direction_estimation_methods()
            m = dem.retrieve_features('all', t)
            direction_estimation_features = np.asarray([m[n] for n in m])
            
            music_features = music(t)
            #display(music_features)
            burgs = arburg(t, len(t))
            cwt = continuous_wt(t)
            dwt = discrete_wt(t)
            # pad dwt values with zeros so that size matches with other feature vectors
            dwt = list(dwt[0]) + list(np.zeros(len(dwt[0]))) 
            #sf = np.asarray([fft, inverse_fft, ywm, burgs, cwt[0], dwt], dtype=np.float32)  
            
            
            
            
            
            qmf_features = pywt.qmf(t) # Returns the Quadrature Mirror Filter(QMF).
            ofb_features = pywt.orthogonal_filter_bank(t) # Orthogonal Filter Banks; returns The orthogonal filter bank of the input scaling filter in the order : 1] Decomposition LPF 2] Decomposition HPF 3] Reconstruction LPF 4] Reconstruction HPF
            #display(ofb_features[3])

            # inverse stationary wavelet transforms
            coeffs = pywt.swt(t, 'db2', level=2)
            iswt_features  = pywt.iswt(coeffs, 'db2')

            #coeffs = pywt.dwt2(test, 'haar')
            swt_features = pywt.swt(t, 'haar') #stationary wavelet transforms -- can use 'level' parameter to limit output --using only [0, 2, 4]
            #swt_f = [swt_features[0][0], swt_features[1][0], swt_features[2][0]]#, swt_features[4])
            #display(qmf_features)
            sf = np.asarray([fft, inverse_fft, ywm, music_features, burgs, cwt[0], dwt, 
                             qmf_features, iswt_features, ofb_features[0], ofb_features[1], ofb_features[2], ofb_features[3],
                             swt_features[0][0], swt_features[0][1], swt_features[1][0], swt_features[1][1]]
                            , dtype=np.float32)   
            
            
            
            
            sf = np.append(sf, direction_estimation_features, axis=0)
            sample_features.extend([sf])
        all_features.append(np.asarray(sample_features, dtype=np.float32))
    return all_features


hc_features = extract_features(hc_data)
sz_features = extract_features(sz_data)

C:\Users\marit\Anaconda3\lib\site-packages\ipykernel_launcher.py:141: ComplexWarning: Casting complex values to real discards the imaginary part


In [20]:
#!pip install scipy==1.2 --upgrade
np.asarray(hc_features).shape

(350, 20, 22, 12)

In [ ]:
#fv_test = hc_data[0][0][0]#.reshape(-1, 1)
#fv_test.reshape(-1, 1)


np.linspace(-1, 1, 30, endpoint=False)
test = np.linspace(-1, 1, 30, endpoint=False)
#music(test)

np.asarray(hc_features).shape

In [21]:
# save ndarrays to .npy files
def save_ndarray(x, file_name):
    np.save(file_name, x)
    
save_ndarray(hc_features, 'hc_features_22')
save_ndarray(sz_features, 'sz_features_22')


In [ ]:
hc

In [ ]:
## End of implementation code  

print('Printing environment settings')

from platform import python_version
print('\nPython version: ', python_version())
print('\nInstalled modules:\n')

!pip freeze